## Random Forests vs. Decision Trees
Summary:

    1. Random forest algorithms sample the data and build several smaller, simpler decision trees.
    
    2. Run efficiently on large datasets.
    
    3. Are robust to outliers and nonlinear data.

### Import dependencies and load data

In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Loading data
file_path = Path("../Resources/loans_data_encoded.csv")
df_loans = pd.read_csv(file_path)
df_loans.head()

,amount,term,age,bad,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male,month_num
0,1000,30,45,0,0,1,0,0,0,1,6
1,1000,30,50,0,1,0,0,0,1,0,7
2,1000,30,33,0,1,0,0,0,1,0,8
3,1000,15,27,0,0,0,0,1,0,1,9
4,1000,30,28,0,0,0,0,1,1,0,10


## Preprocess the data

In [3]:
# Define the features set (X).
X = df_loans.copy()
X = X.drop("bad", axis=1)
X.head()

,amount,term,age,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male,month_num
0,1000,30,45,0,1,0,0,0,1,6
1,1000,30,50,1,0,0,0,1,0,7
2,1000,30,33,1,0,0,0,1,0,8
3,1000,15,27,0,0,0,1,0,1,9
4,1000,30,28,0,0,0,1,1,0,10


In [10]:
# Define the target set (y) - note: ravel() performs
# the same as the values attribute
y = df_loans["bad"].ravel()
y[:5]

array([0, 0, 0, 0, 0])

### Split data into training and testing sets

In [11]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(375, 10)
(125, 10)
(375,)
(125,)


### Scale the data

In [13]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [14]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Fit the Random Forest Model

In [19]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [20]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

### Make Predictions Using the Testing Data

In [21]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1])

### Evaluate the Model

In [22]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,50,34
Actual 1,25,16


In [23]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.528

In [24]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,50,34
Actual 1,25,16


Accuracy Score : 0.528
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        84
           1       0.32      0.39      0.35        41

    accuracy                           0.53       125
   macro avg       0.49      0.49      0.49       125
weighted avg       0.55      0.53      0.54       125



Analysis: accuracy score and F1 score is too low, and therefore random forest model is not good at classifying fraudulent loan applications.

### Rank the Importance of Features
Note: random forest algorithms allows us to rank the features by their importance; this allows us to see which features have the most impact on the decision made by the random forest.

In [25]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05575596, 0.08274963, 0.42718881, 0.02027503, 0.02434651,
       0.00293021, 0.02662008, 0.01965243, 0.0195016 , 0.32097974])

In [26]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.42718880810849064, 'age'),
 (0.32097973949900543, 'month_num'),
 (0.08274963450844049, 'term'),
 (0.05575595964447335, 'amount'),
 (0.02662008012279912, 'education_college'),
 (0.024346506435266257, 'education_High School or Below'),
 (0.020275026448816385, 'education_Bachelor'),
 (0.01965243274729426, 'gender_female'),
 (0.019501604011298697, 'gender_male'),
 (0.00293020847411546, 'education_Master or Above')]

Note: this shows which features (i.e. columns) of the loan application influenced the decision. Features most high ranked are age and month_num. To improve the model, we can drop some of the lower ranked features.

## Improving the model

In [28]:
# Define the features set (X).
X = df_loans.copy()
X = X.drop(["bad", "gender_female", "gender_male", "education_Master or Above"], axis=1)
X.head()

,amount,term,age,education_Bachelor,education_High School or Below,education_college,month_num
0,1000,30,45,0,1,0,6
1,1000,30,50,1,0,0,7
2,1000,30,33,1,0,0,8
3,1000,15,27,0,0,1,9
4,1000,30,28,0,0,1,10


In [29]:
# Define the target set (y) - note: ravel() performs
# the same as the values attribute
y = df_loans["bad"].ravel()
y[:5]

array([0, 0, 0, 0, 0])

In [30]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [31]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(375, 7)
(125, 7)
(375,)
(125,)


In [32]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [33]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [35]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [36]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1])

In [37]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,53,31
Actual 1,25,16


In [38]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.552

In [39]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,53,31
Actual 1,25,16


Accuracy Score : 0.552
Classification Report
              precision    recall  f1-score   support

           0       0.68      0.63      0.65        84
           1       0.34      0.39      0.36        41

    accuracy                           0.55       125
   macro avg       0.51      0.51      0.51       125
weighted avg       0.57      0.55      0.56       125



Note: by dropping less important categories, accuracy of the random forest increased to 55%. Still not a significant improvement to the accuracy of the model. 